<a href="https://colab.research.google.com/github/Harsh-Mathur-1503/genrative_ai/blob/main/gen_ai_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torch==1.13.1 torchdata==0.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 54.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 1.13.1 which is 

In [3]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [33]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In this use case, we will be generating summary of a dialogue with the pre-trained Large Language Model FLAN-75 from Hugging Face.

Now we'll upload some simple dialogue from **DialogSum** Hugging Face dataset which consists of 10000+ dialogues manually labelled summaries and topics.

In [5]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

**Print a couple of dialogues with baseline summaries**

In [7]:
example_indices = [40,200]
dash_line = '-'.join('' for x in range(100))

for i,index in enumerate(example_indices):
  print(dash_line)
  print('EXAMPLE ',i+1)
  print(dash_line)
  print('INPUT_DIALOGUE : ')
  print(dataset['test'][index]['dialogue'])
  print(dash_line)
  print('BASELINE_SUMMARY : ')
  print(dataset['test'][index]['summary'])
  print(dash_line)
  print()

---------------------------------------------------------------------------------------------------
EXAMPLE  1
---------------------------------------------------------------------------------------------------
INPUT_DIALOGUE : 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE_SUMMARY : 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
EXAMP

Load the FLAN-T5 Model, creating an instance of **AutoModelForSeq2SeqLM** class with **.from_pretrained()** method.

In [8]:
model_name = "google/flan-t5-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

To perform encoding and decoding , I need to work with tokenized form.
**Tokenisation** is a process of splitting texts into smaller units that can be processed by LLM models.

Parameter **use_fast** switches on fast tokenizer. At this stage, there is no need to go into the details of that but you can find the tokenizer parameters in the documentation.

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [10]:
sentence = "What time is it Tom ?"
sentence_encoded = tokenizer(sentence,return_tensors="pt")

sentence_decoded = tokenizer.decode(sentence_encoded['input_ids'][0],skip_special_tokens=True)
print("ENCODED SENTENCE : ")
print(sentence_encoded["input_ids"][0])
print("\n DECODED SENTENCE : ")
print(sentence_decoded)

ENCODED SENTENCE : 
tensor([ 363,   97,   19,   34, 3059,    3,   58,    1])

 DECODED SENTENCE : 
What time is it Tom?


Observing how well our LLM performs without any prompt engineering.

In [11]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors="pt")

    outputs = tokenizer.decode(
        model.generate(inputs["input_ids"], max_new_tokens=50)[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print(f'EXAMPLE {i + 1}')
    print(dash_line)
    print(f'INPUT_DIALOGUE:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE_HUMAN_SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL_GENERATED_SUMMARY_WITHOUT_PROMPT_ENGINEERING:\n{outputs}\n')


---------------------------------------------------------------------------------------------------
EXAMPLE 1
---------------------------------------------------------------------------------------------------
INPUT_DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE_HUMAN_SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL_GENERATED_SUMMARY_WITHOUT_PROMPT_ENGINEERING:
#Person1#: I'm afraid I'm late.

--------------------

Summarize the dialogues using Prompt Engineering -


*   **Zero shot inference** -  Take a dialogue and convert it into instruction prompt



In [12]:
for i,index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
Summarize the following conversation.
{dialogue}
Summary :
"""

  # input the constructed prompt instead of the dialogue
  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = tokenizer.decode(
      model.generate(inputs["input_ids"], max_new_tokens=50)[0],
      skip_special_tokens=True
  )

  print(dash_line)
  print(f'EXAMPLE {i + 1}')
  print(dash_line)
  print(f'INPUT_DIALOGUE:\n{dialogue}')
  print(dash_line)
  print(f'BASELINE_HUMAN_SUMMARY:\n{summary}')
  print(dash_line)
  print(f'MODEL_GENERATED_SUMMARY_WITH_ZERO_SHOT_INFERENCE : \n{outputs}\n')


---------------------------------------------------------------------------------------------------
EXAMPLE 1
---------------------------------------------------------------------------------------------------
INPUT_DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE_HUMAN_SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL_GENERATED_SUMMARY_WITH_ZERO_SHOT_INFERENCE : 
Tom has to get off the train at nine thirty.

-------

In [13]:
for i,index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
Dialogue :
{dialogue}
What was going on ?
"""

  # input the constructed prompt instead of the dialogue
  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = tokenizer.decode(
      model.generate(inputs["input_ids"], max_new_tokens=50)[0],
      skip_special_tokens=True
  )

  print(dash_line)
  print(f'EXAMPLE {i + 1}')
  print(dash_line)
  print(f'INPUT_DIALOGUE:\n{dialogue}')
  print(dash_line)
  print(f'BASELINE_HUMAN_SUMMARY:\n{summary}')
  print(dash_line)
  print(f'MODEL_GENERATED_SUMMARY_WITH_ZERO_SHOT_INFERENCE : \n{outputs}\n')


---------------------------------------------------------------------------------------------------
EXAMPLE 1
---------------------------------------------------------------------------------------------------
INPUT_DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE_HUMAN_SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL_GENERATED_SUMMARY_WITH_ZERO_SHOT_INFERENCE : 
Person1 is late for his train. He has to get off now.

Summarize the dialogues using Prompt Engineering -


*   **One shot inference** -  Let's build a function that takes a list of example_indices_full, generates a prompt with full examples , then at the end appends the prompt which you want the model to complete (example_index_to_summarize).



In [16]:
def make_prompt(example_indices_full, example_index_to_summarize):
  prompt = ''
  for i in example_indices_full:
    dialogue = dataset['test'][i]['dialogue']
    summary = dataset['test'][i]['summary']
    # The stop sequence `{summary}\n\n\n` is important for FLAN-T5. Other models may have their own stop sequences.
    prompt += f"""
Dialogue :
{dialogue}

What was going on ?
{summary}
"""
  dialogue = dataset['test'][example_index_to_summarize]['dialogue']
  prompt += f"""
Dialogue :
{dialogue}

What was going on ?
"""
  return prompt

**Construct the prompt to perform one-shot inference**

In [17]:
example_indices_full=[40]
example_index_to_summarize=200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)
print(one_shot_prompt)


Dialogue : 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on ?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

Dialogue : 
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also n

Now pass this prompt to perform the one shot inference

In [19]:
summary = dataset['test'][example_index_to_summarize]['summary']
inputs = tokenizer(one_shot_prompt, return_tensors="pt")
outputs = tokenizer.decode(
    model.generate(inputs["input_ids"], max_new_tokens=50)[0],
    skip_special_tokens=True
)
print(dash_line)
print(f"BASELINE_HUMAN_SUMMARY:\n{summary}\n")
print(dash_line)
print(f"MODEL GENERATION_ONE_SHOT:\n{outputs}\n")


---------------------------------------------------------------------------------------------------
BASELINE_HUMAN_SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION_ONE_SHOT:
Person2 is considering upgrading her system. She needs a faster processor, a faster hard disc, more memory and a faster modem. She doesn't have a CD-ROM drive.



Summarize the dialogues using Prompt Engineering -


*   **Few shot inference** -  we'll provide two more full dialogue-summary to our model in prompt.



In [20]:
example_indices=[40,80,120]
example_index_to_summarize=200

few_shot_prompt = make_prompt(example_indices, example_index_to_summarize)
print(few_shot_prompt)


Dialogue : 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on ?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

Dialogue : 
#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set.

Now pass this prompt to perform few shot inference :

In [22]:
summary = dataset['test'][example_index_to_summarize]['summary']
inputs = tokenizer(few_shot_prompt, return_tensors="pt")
outputs = tokenizer.decode(
    model.generate(inputs["input_ids"], max_new_tokens=50)[0],
    skip_special_tokens=True
)

print(dash_line)
print(f"BASELINE_HUMAN_SUMMARY:\n{summary}\n")
print(dash_line)
print(f"MODEL GENERATION_FEW_SHOT:\n{outputs}\n")

---------------------------------------------------------------------------------------------------
BASELINE_HUMAN_SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION_FEW_SHOT:
Person2 wants to upgrade her system. She needs a faster processor, more memory, a faster modem and a CD-ROM drive.



In this case few shot did not perform that much better than one shot inference.
Add anything above 5 or 6 shot will typically not help much,either.

**Genrative Configuration Parameters for Inference**

We'll use GenerateConfig class

In [35]:
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50,do_sample=True,temperature=0.1)
generation_config = GenerationConfig(max_new_tokens=50,do_sample=True,temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50,do_sample=True,temperature=1.0)

In [36]:
summary = dataset['test'][example_index_to_summarize]['summary']
inputs = tokenizer(few_shot_prompt, return_tensors="pt")
outputs = tokenizer.decode(
    model.generate(inputs["input_ids"], generation_config=generation_config)[0],
    skip_special_tokens=True
)

print(dash_line)
print(f"BASELINE_HUMAN_SUMMARY:\n{summary}\n")
print(dash_line)
print(f"MODEL GENERATION_FEW_SHOT:\n{outputs}\n")

---------------------------------------------------------------------------------------------------
BASELINE_HUMAN_SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION_FEW_SHOT:
#Person1 suggests adding a painting program to the system. #Person2 wants to upgrade the hardware. #Person1 suggests adding a CD-ROM drive. most new software programs are coming out on CDs.

